In [ ]:
import pandas as pd

df_1 = pd.read_csv('cons_trans_translate.csv',delimiter='\t')
df_1 = df_1.fillna('')

df_1_answer = df_1.loc[df_1['reponse2'] != '']
df_1_answer['answer'] = df_1_answer['reponse2'] + ' |receiver_place: ' + df_1_answer['reciever_place'] +  ' |time_process: ' +  df_1_answer['time_processing'] + ' |cost: ' + df_1_answer['cost'] + ' |link: ' + df_1_answer['link']
df_1_answer = df_1_answer[['idx','answer']]
df_1_question = df_1[['idx','procs']]

df_1_final = df_1_question.merge(df_1_answer, on='idx', how='left')
df_1_final = df_1_final.fillna('')
df_1_final = df_1_final.loc[df_1_final['idx'] != '']

df_1_final.columns = ['idx','question','answer']
df_1_final.to_csv('cons_trans_model.csv')
df_1_final['messages'] = df_1_final.apply(lambda x:  [{'role': 'system','content':'cons_trans'},{'role': 'user', 'content' : x['question']}, {'role':'assistant', 'content':x['answer']}], axis = 1)
df_1_json = df_1_final[['messages']]
df_1_json.to_json('cons_trans_model.jsonl',orient='records',lines=True)

In [ ]:
df_2 = pd.read_csv('eco_trad_translate.csv',delimiter='\t')
df_2 = df_2.fillna('')
# print(df_2)
df_2_answer = df_2.loc[df_2['reponse2'] != '']
# print(df_2_answer)
df_2_answer['answer'] = df_2_answer['reponse2'] + ' |receiver_place: ' + df_2_answer['reciever_place'] +  ' |time_process: ' +  df_2_answer['time_processing'] + ' |cost: ' + df_2_answer['cost'] + ' |link: ' + df_2_answer['link']
df_2_answer = df_2_answer[['idx','answer']]
# print(df_2_answer)

df_2_question = df_2[['idx','procs']]

df_2_final = df_2_question.merge(df_2_answer, on='idx', how='left')
df_2_final = df_2_final.fillna('')
df_2_final = df_2_final.loc[df_2_final['idx'] != '']

df_2_final.columns = ['idx','question','answer']
df_2_final.to_csv('eco_trad_model.csv')

df_2_final['messages'] = df_2_final.apply(lambda x:  [{'role': 'system','content':'cons_trans'},{'role': 'user', 'content' : x['question']}, {'role':'assistant', 'content':x['answer']}], axis = 1)
# print(df_1_final)
df_2_json = df_2_final[['messages']]
df_2_json.to_json('eco_trad_model.jsonl',orient='records',lines=True)

In [ ]:
df_3 = pd.read_csv('res_env_translate.csv',delimiter='\t')
df_3 = df_3.fillna('')
# print(df_3)
df_3_answer = df_3.loc[df_3['reponse2'] != '']
# print(df_3_answer)
df_3_answer['answer'] = df_3_answer['reponse2'] + ' |receiver_place: ' + df_3_answer['reciever_place'] +  ' |time_process: ' +  df_3_answer['time_processing'] + ' |cost: ' + df_3_answer['cost'] + ' |link: ' + df_3_answer['link']
df_3_answer = df_3_answer[['idx','answer']]
# print(df_3_answer)

df_3_question = df_3[['idx','procs']]

df_3_final = df_3_question.merge(df_3_answer, on='idx', how='left')
df_3_final = df_3_final.fillna('')
df_3_final = df_3_final.loc[df_3_final['idx'] != '']

df_3_final.columns = ['idx','question','answer']
df_3_final.to_csv('res_env_model.csv')

df_3_final['messages'] = df_3_final.apply(lambda x:  [{'role': 'system','content':'cons_trans'},{'role': 'user', 'content' : x['question']}, {'role':'assistant', 'content':x['answer']}], axis = 1)
# print(df_1_final)
df_3_json = df_3_final[['messages']]
df_3_json.to_json('res_env_model.jsonl',orient='records',lines=True)

In [ ]:
import yaml

with open('nlu_greeting.yml', 'r') as file:
    prime_service = yaml.safe_load(file)
text = []
for i in prime_service['responses'].keys():
    for j in prime_service['responses'][i]:
        text.append({'intent': i, 'answer': j['text']})
df_nlu = pd.DataFrame(prime_service['nlu'])
df_response = pd.DataFrame(text)
df_response['intent'] = df_response['intent'].apply(lambda x: x.replace('utter_',''))

df_total = df_nlu.merge(df_response, on='intent', how='left')
print(df_total)

df_total.columns = ['idx','question','answer']
df_total.to_csv('greeting_model.csv')

df_total['messages'] = df_total.apply(lambda x:  [{'role': 'system','content':'greeting'},{'role': 'user', 'content' : x['question']}, {'role':'assistant', 'content':x['answer']}], axis = 1)
# print(df_1_final)
df_total = df_total[['messages']]
df_total.to_json('greeting_model.jsonl',orient='records',lines=True)

In [ ]:
%env OPENAI_API_KEY=<YOUR API KEY>

In [ ]:
from train_gpt_utilities import TrainGPT
trainer = TrainGPT()


trainer.create_file("greeting_model.jsonl")
trainer.start_training()

trainer.create_file("cons_trans_model.jsonl")
trainer.start_training()

trainer.create_file("eco_trad_model.jsonl")
trainer.start_training()

trainer.create_file("res_env_model.jsonl")
trainer.start_training()


In [ ]:
jobs = trainer.list_jobs()